In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [90]:
img = cv.imread("C:/Users/Victor Daugaard/Desktop/White hair test/PAT_79_120_739.png")
mask = cv.imread("C:/Users/Victor Daugaard/Desktop/White hair test/PAT_79_120_739_mask.png")
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Contrastor not needed

In [ ]:
# NOT NEEDED!!!
def contraster(img,contrast, reverse=False):
    # Takes a contrast vector in bgr
    
    # Copy the image and turn it into floats
    # Makes it possible to work with 0-1 range
    tmp = img.copy()
    tmp = tmp.astype(float)
    
    # Find dimensions of image to loop through all pixels
    dimensions = list(np.shape(tmp))[:2]
    
    # Loop through all pixels
    for x in range(dimensions[0]):
        for y in range(dimensions[1]):
            # bgr color, b=0   g = 1   r = 2
            
            # Convert range from 0-255 to 0-1
            tmp[x,y,2] = tmp[x,y,2] / 255
            tmp[x,y,1] = tmp[x,y,1] / 255
            tmp[x,y,0] = tmp[x,y,0] / 255
            
            # Apply contrasting
            tmp[x,y,2] = (tmp[x,y,2]-0.5) * contrast[2] + 0.5
            tmp[x,y,1] = (tmp[x,y,1]-0.5) * contrast[1] + 0.5
            tmp[x,y,0] = (tmp[x,y,0]-0.5) * contrast[0] + 0.5
            
            # Reverse colors if reverse=True
            if reverse == True:
                tmp[x,y,2] = tmp[x,y,2] * -1 + 1
                tmp[x,y,1] = tmp[x,y,1] * -1 + 1
                tmp[x,y,0] = tmp[x,y,0] * -1 + 1
            
            # Convert range from 0-1 to 0-255
            tmp[x,y,2] = tmp[x,y,2] * 255
            tmp[x,y,1] = tmp[x,y,1] * 255
            tmp[x,y,0] = tmp[x,y,0] * 255
    
    # Change the color value type to uint8, clamp between 0 - 255, and return
    # Maybe consider a tonemapper instead of clamp
    return np.clip(tmp.astype(np.uint8), 0, 255)

# Saturator not needed

In [ ]:
# NOT NEEDED!!!!!!!!
def saturator(img,saturation):
    # Takes a saturation vector in bgr
    
    # Copy the image and turn it into floats
    # Makes it possible to work with 0-1 range
    tmp = img.copy()
    tmp = tmp.astype(float)
    
    # Find dimensions of image to loop through all pixels
    dimensions = list(np.shape(tmp))[:2]
    
    # Loop through all pixels
    for x in range(dimensions[0]):
        for y in range(dimensions[1]):
            # bgr color, b=0   g = 1   r = 2
            
            # Convert range from 0-255 to 0-1
            tmp[x,y,2] = tmp[x,y,2] / 255
            tmp[x,y,1] = tmp[x,y,1] / 255
            tmp[x,y,0] = tmp[x,y,0] / 255
            
            # Calculate luminance of pixel
            luminance = [0,0,0]
            luminance[2] = tmp[x,y,2] * 0.299
            luminance[1] = tmp[x,y,1] * 0.574
            luminance[0] = tmp[x,y,0] * 0.127
            
            # Apply saturation
            tmp[x,y,2] = tmp[x,y,2] * saturation[2] + luminance[2] * (1-saturation[2])
            tmp[x,y,1] = tmp[x,y,1] * saturation[1] + luminance[1] * (1-saturation[1])
            tmp[x,y,0] = tmp[x,y,0] * saturation[0] + luminance[0] * (1-saturation[0])
            
            # Convert range from 0-1 to 0-255
            tmp[x,y,2] = tmp[x,y,2] * 255
            tmp[x,y,1] = tmp[x,y,1] * 255
            tmp[x,y,0] = tmp[x,y,0] * 255
    
    # Change the color value type to uint8, clamp between 0 - 255, and return
    # Maybe consider a tonemapper instead of clamp
    return np.clip(tmp.astype(np.uint8), 0, 255)

In [5]:
def removeHair(img_org, img_gray, kernel_size=25, threshold=10, radius=3):
    # kernel for the morphological filtering
    kernel = cv.getStructuringElement(cv.MORPH_CROSS, (kernel_size, kernel_size))

    # perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv.morphologyEx(img_gray, cv.MORPH_BLACKHAT, kernel)

    # intensify the hair countours in preparation for the inpainting algorithm
    _, thresh = cv.threshold(blackhat, threshold, 255, cv.THRESH_BINARY)

    # inpaint the original image depending on the mask
    img_out = cv.inpaint(img_org, thresh, radius, cv.INPAINT_TELEA)

    return blackhat, thresh, img_out

In [ ]:
def removeHairWhite(img_org, kernel_size=25, threshold=10, radius=3):

    # Color reverser start
    # Copy the image and turn it into floats
    # Makes it possible to work with 0-1 range
    img_reverse = img.copy()
    img_reverse = img_reverse.astype(float)
    
    # Find dimensions of image to loop through all pixels
    dimensions = list(np.shape(img_reverse))[:2]
    
    # Loop through all pixels
    for x in range(dimensions[0]):
        for y in range(dimensions[1]):
            # bgr color, b=0   g = 1   r = 2
            
            # Convert range from 0-255 to 0-1
            img_reverse[x,y,2] = img_reverse[x,y,2] / 255
            img_reverse[x,y,1] = img_reverse[x,y,1] / 255
            img_reverse[x,y,0] = img_reverse[x,y,0] / 255
            
            # Reverse colors to make black -> white and white -> black
            img_reverse[x,y,2] = img_reverse[x,y,2] * -1 + 1
            img_reverse[x,y,1] = img_reverse[x,y,1] * -1 + 1
            img_reverse[x,y,0] = img_reverse[x,y,0] * -1 + 1
            
            # Convert range from 0-1 to 0-255
            img_reverse[x,y,2] = img_reverse[x,y,2] * 255
            img_reverse[x,y,1] = img_reverse[x,y,1] * 255
            img_reverse[x,y,0] = img_reverse[x,y,0] * 255
    
    img_reverse = np.clip(img_reverse.astype(np.uint8), 0, 255)
    # Color reverser end
    
    gray_reverse = cv.cvtColor(img_reverse, cv.COLOR_BGR2GRAY)
    
    # kernel for the morphological filtering
    kernel = cv.getStructuringElement(cv.MORPH_CROSS, (kernel_size, kernel_size))

    # perform the blackHat filtering on the grayscale image to find the hair countours
    blackhat = cv.morphologyEx(gray_reverse, cv.MORPH_BLACKHAT, kernel)

    # intensify the hair countours in preparation for the inpainting algorithm
    _, thresh = cv.threshold(blackhat, threshold, 255, cv.THRESH_BINARY)

    # inpaint the original image depending on the mask
    img_out = cv.inpaint(img_org, thresh, radius, cv.INPAINT_TELEA)

    return blackhat, thresh, img_out

In [ ]:
# White hair removal
_, _, img_out1 = removeHairWhite(img)

# Black hair removal
_, _, img_out2 = removeHair(img,gray)

# Black hair removal, then white hair removal
# _, _, img_out3 = removeHairWhite(img_out2)

# White then black
# _, _, img_out4 = removeHair(img_out1, gray)

# Creating masked images
# masked = cv.bitwise_and(img, mask)
# masked_removed = cv.bitwise_and(img_out1, mask)

In [94]:
cv.imshow("Image", img)
cv.imshow("White hair removed image", img_out1)
cv.imshow("Black hair removed image", img_out2)
# cv.imshow("Black then white hair removed image", img_out3)
# cv.imshow("White then black hair removed image", img_out4)
cv.waitKey(0)
cv.destroyAllWindows()

"White then black" is better than "black then white".

It changes the image drastically either way, so not sure if it's very good to do that.